<a href="https://colab.research.google.com/github/chrisdepallan/chris_projects/blob/main/tensorflow_twt_sentiment_analysis_multiclass_classi.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import LabelEncoder
import numpy as np
from tensorflow.keras.layers import Embedding, LSTM, Dense, GlobalMaxPooling1D

In [ ]:
dataset=pd.read_csv('tweet_emotions.csv')
dataset

,tweet_id,sentiment,content
0,1956967341,empty,@tiffanylue i know i was listenin to bad habi...
1,1956967666,sadness,Layin n bed with a headache ughhhh...waitin o...
2,1956967696,sadness,Funeral ceremony...gloomy friday...
3,1956967789,enthusiasm,wants to hang out with friends SOON!
4,1956968416,neutral,@dannycastillo We want to trade with someone w...
...,...,...,...
39995,1753918954,neutral,@JohnLloydTaylor
39996,1753919001,love,Happy Mothers Day All my love
39997,1753919005,love,Happy Mother's Day to all the mommies out ther...
39998,1753919043,happiness,@niariley WASSUP BEAUTIFUL!!! FOLLOW ME!! PEE...


In [ ]:
x=dataset['content']
y=dataset['sentiment']
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2)


In [ ]:
#tokenizing


tokenizer=Tokenizer()
tokenizer.fit_on_texts(x_train)

x_train_sequences=tokenizer.texts_to_sequences(x_train)
x_test_sequences=tokenizer.texts_to_sequences(x_test)
# x_train_sequences
vocab_size = len(tokenizer.word_index) + 1
max=255
x_train_padded = pad_sequences(x_train_sequences,maxlen=max, padding='post')

x_test_padded = pad_sequences(x_test_sequences,maxlen=max,  padding='post')


In [ ]:
#label_encoder
label_encode=LabelEncoder()
y_train_encoded=label_encode.fit_transform(y_train)
y_test_encoded=label_encode.transform(y_test)
y_train_encoded = np.array(y_train_encoded)
y_test_encoded = np.array(y_test_encoded)

In [ ]:
embedding_dim

41981

In [ ]:
model_00=tf.keras.Sequential(
    # [
    # tf.keras.layers.Embedding(input_dim=len(tokenizer.word_index) + 1, output_dim=max, input_length=max),
    # tf.keras.layers.Dense(256,activation='relu'),
    # tf.keras.layers.Dense(128,activation='relu'),
    # tf.keras.layers.Dense(64,activation='relu'),
    # tf.keras.layers.Dense(13,activation='softmax')
# ]
                             )
model_00.add(Embedding(input_dim=vocab_size, output_dim=embedding_dim, input_length=max))
model_00.add(LSTM(64, return_sequences=True))
model_00.add(GlobalMaxPooling1D())
model_00.add(Dense(32, activation='relu'))
model_00.add(Dense(len(dataset['sentiment'].unique()), activation='softmax'))
model_00.compile(
    loss=tf.keras.losses.SparseCategoricalCrossentropy(),
    optimizer=tf.keras.optimizers.Adam(),
    metrics=['accuracy']
)
# model_00.build()
# model_00.summary()
model_00.fit(x_train_padded,y_train_encoded,epochs=1)

  36/1000 [>.............................] - ETA: 17:24 - loss: 2.2255 - accuracy: 0.2222

In [ ]:
np.shape(x_train_padded),np.shape(y_train_encoded)

((32000, 255), (32000,))

In [ ]:
# len(y_train_encoded)
# len(x_train_padded)
len(dataset['sentiment'].unique())

13

In [ ]:
np.array(x_train_padded)==x_train_padded

array([[ True,  True,  True, ...,  True,  True,  True],
       [ True,  True,  True, ...,  True,  True,  True],
       [ True,  True,  True, ...,  True,  True,  True],
       ...,
       [ True,  True,  True, ...,  True,  True,  True],
       [ True,  True,  True, ...,  True,  True,  True],
       [ True,  True,  True, ...,  True,  True,  True]])

NameError: ignored

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np

# Load the dataset
dataset = pd.read_csv('tweet_emotions.csv')

# Split the dataset into features (x) and labels (y)
x = dataset['content']
y = dataset['sentiment']

# Create a tokenizer and fit it on the training data
tokenizer = Tokenizer()
tokenizer.fit_on_texts(x)

# Convert text to sequences
x_sequences = tokenizer.texts_to_sequences(x)

# Vocabulary size (total number of unique words in the corpus)
vocab_size = len(tokenizer.word_index) + 1

# Maximum sequence length (you can set this to the maximum tweet length in your dataset)
max_sequence_length = 100

# Pad sequences to have the same length
x_padded = pad_sequences(x_sequences, maxlen=max_sequence_length, padding='post')

# Convert sentiment labels to numerical representation
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

# Split the data into training and testing sets
x_train, x_test, y_train, y_test = train_test_split(x_padded, y_encoded, test_size=0.2, random_state=42)
embedding_dim = 100
# Model architecture
model_00 = tf.keras.Sequential([
    tf.keras.layers.Embedding(input_dim=vocab_size, output_dim=embedding_dim, input_length=max_sequence_length),
    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(len(label_encoder.classes_), activation='softmax')  # Use the number of unique classes for the output layer
])

model_00.compile(
    loss='sparse_categorical_crossentropy',
    optimizer=tf.keras.optimizers.Adam(),
    metrics=['accuracy']
)

print(model_00.summary())

# Now, we can proceed with training the model
model_00.fit(x_train, y_train, epochs=10, batch_size=32, validation_split=0.1)


Model: "sequential_20"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_14 (Embedding)    (None, 100, 100)          4899800   
                                                                 
 dense_86 (Dense)            (None, 100, 256)          25856     
                                                                 
 dense_87 (Dense)            (None, 100, 128)          32896     
                                                                 
 dense_88 (Dense)            (None, 100, 64)           8256      
                                                                 
 dense_89 (Dense)            (None, 100, 13)           845       
                                                                 
Total params: 4,967,653
Trainable params: 4,967,653
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/10


ValueError: ignored